In [58]:
import gensim
from configparser import ConfigParser
from pymongo import MongoClient
import re
import string
import warnings
import os
from tabulate import tabulate

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    import spacy
    nlp = spacy.load('en')
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from collections import defaultdict, Counter

os.chdir('/Users/tmoeller/ds/twitter-project/partisan-tweets')
import src.data.aws_ec2_functions as aws

In [19]:
# Make sure AWS Ec2 Instance is running and get public IP address
instance = aws.fetch_instances()[0]

if aws.instance_state(instance) != 'running':
    print('Starting {} instance now'.format(instance.public_ip_address))
    aws.start_instance(instance, safety=False)

In [20]:
punctuations = string.punctuation
config = ConfigParser()
config.read('config.ini')

['config.ini']

In [21]:
client = MongoClient("mongodb://{}:{}@{}/{}".format(
                        config.get('MongoConfig', 'user'),
                        config.get('MongoConfig', 'password'),
                        instance.public_ip_address,
                        config.get('MongoConfig', 'db'),
                        int(config.get('MongoConfig', 'port'))))

db = client.twitter_db

In [22]:
def fetch_train_test(train_group, test_group):
    cursor = db.train_test_dict.find({'group': train_group}, { 'screen_names': 1, '_id': 0 })
    train_group = [doc for doc in cursor]

    cursor = db.train_test_dict.find({'group': test_group}, { 'screen_names': 1, '_id': 0 })
    test_group = [doc for doc in cursor]

    cursor = db.fav_tweets_dict.find( { "name": { "$in": train_group[0]['screen_names'] } } )
    train = [doc for doc in cursor]

    cursor = db.legislator_tweets_dict.find( { "name": { "$in": test_group[0]['screen_names'] } } )
    test = [doc for doc in cursor]

    return train, test


def unlist_tweets(list_tweet_dicts):
    tweets_labels = []

    for dict in list_tweet_dicts:
        for tweet in dict['tweets']:
            tweets_labels.append([tweet, dict['party']])

    return tweets_labels


def clean_tweet(tweet):
    """
    Function to remove urls, numbers and punctuation, and make lowercase
    """
    no_url = re.sub(r'http\S+', '', tweet)
    clean = re.sub(r'[^\w\s]', '', no_url)

    result = ''.join([str(i).replace('\n', ' ').lower() for i in clean if not i.isdigit()])

    return result


def spacy_tokenizer(tweet):
    """
    Utility function to remove stopwords, ignore pronouns and tokenize words before vectorizing
    """
    doc = nlp(tweet)
    tokens = [token.orth_ for token in doc if not token.is_stop]

    return tokens

In [23]:
train_a, test_b = fetch_train_test('train_a', 'test_b')

train_data = unlist_tweets(train_a)
test_data = unlist_tweets(test_b)

In [29]:
print(len(train_data))
print(len(test_data))

143531
13610


In [60]:
clean_train = [clean_tweet(tweet[0]) for tweet in train_data]
clean_test = [clean_tweet(tweet[0]) for tweet in test_data]

y_train = [1 if tweet[1]=='R' else 0 for tweet in train_data]
y_test = [1 if tweet[1]=='R' else 0 for tweet in test_data]

In [61]:
Counter(y_train)

Counter({0: 64598, 1: 78933})

In [25]:
tokenized_train = [spacy_tokenizer(tweet) for tweet in clean_train]
tokenized_test = [spacy_tokenizer(tweet) for tweet in clean_test]

In [26]:
# let X be a list of tokenized texts (i.e. list of lists of tokens)
model = gensim.models.Word2Vec(tokenized_train, size=100)

In [32]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/Users/tmoeller/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [37]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.items())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.items())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [48]:
bern_nb = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("bernoulli nb", BernoulliNB())])
bern_nb_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("bernoulli nb", BernoulliNB())])

In [38]:
etree_w2v = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])
etree_w2v_tfidf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])

In [62]:
all_models = [
    ("bern_nb", bern_nb),
    ("bern_nb_tfidf", bern_nb_tfidf),
    ("w2v", etree_w2v),
    ("w2v_tfidf", etree_w2v_tfidf)
]


unsorted_scores = [(name, cross_val_score(model, clean_train, y_train, cv=3, scoring='roc_auc').mean()) for name, model in all_models]
scores = sorted(unsorted_scores, key=lambda x: -x[1])

In [63]:
print (tabulate(scores, floatfmt=".4f", headers=("model", 'score')))

model            score
-------------  -------
bern_nb         0.5414
bern_nb_tfidf   0.5414
w2v_tfidf       0.5328
w2v             0.5309
